<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Gabriela Mora
- Nombre de alumno 2: Raúl Silva


### **Link de repositorio de GitHub:** `https://github.com/RaulSilvaA/MDS7202-mora-silva`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 48.6 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones.

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except:
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


In [ ]:
df_players = pd.read_csv('/content/drive/MyDrive/lab_progra/stats_players.csv')
pd.set_option('display.max_columns', None)
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,Ball_Control,Dribbling,Marking,Sliding_Tackle,Standing_Tackle,Aggression,Reactions,Interceptions,Vision,Composure,Crossing,Short_Pass,Long_Pass,Acceleration,Speed,Stamina,Strength,Balance,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,5,93,92,22,23,31,63,96,29,85,86,84,83,77,91,92,92,80,63,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,4,95,97,13,26,28,48,95,22,90,94,77,88,87,92,87,74,59,95,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,5,95,96,21,33,24,56,88,36,80,80,75,81,75,93,90,79,49,82,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,4,91,86,30,38,45,78,93,41,84,83,77,83,64,88,77,89,76,60,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,1,48,30,10,11,10,29,85,30,70,70,15,55,59,58,61,44,83,35,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,1,12,11,12,12,13,25,48,12,37,35,12,22,19,25,21,17,52,52,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,1,13,11,11,12,11,15,43,12,27,22,14,27,26,16,19,21,42,45,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,2,44,39,40,29,35,44,42,38,41,35,35,48,42,62,60,64,34,83,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,1,17,13,11,11,14,23,46,12,13,28,15,20,15,42,40,40,61,60,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego.

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país.

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [ ]:
### Código aquí ###
df_players['National_Position'] = df_players['National_Position'].fillna(0)
df_players['es_seleccionado']=np.where(df_players['National_Position']==0, 0, 1)

In [ ]:
df_players.groupby('es_seleccionado')

In [ ]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17588 entries, 0 to 17587
Data columns (total 40 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Name               17588 non-null  object
 1   Nationality        17588 non-null  object
 2   National_Position  17588 non-null  object
 3   Club_Position      17587 non-null  object
 4   Height             17588 non-null  int64 
 5   Weight             17588 non-null  int64 
 6   Preffered_Foot     17588 non-null  object
 7   Age                17588 non-null  int64 
 8   Work_Rate          17588 non-null  object
 9   Weak_foot          17588 non-null  int64 
 10  Skill_Moves        17588 non-null  int64 
 11  Ball_Control       17588 non-null  int64 
 12  Dribbling          17588 non-null  int64 
 13  Marking            17588 non-null  int64 
 14  Sliding_Tackle     17588 non-null  int64 
 15  Standing_Tackle    17588 non-null  int64 
 16  Aggression         17588 non-null  int64

In [ ]:
df_players.Nationality.unique()
df_players.National_Position.unique()
df_players.Club_Position.unique()
df_players.Preffered_Foot.unique()
df_players.Work_Rate.unique()
df_players.es_seleccionado.unique()

array([1, 0])

In [ ]:
df_players['Club_Position'] = df_players['Club_Position'].fillna(0)

In [ ]:
df_players['Nationality'] = df_players['Nationality'].astype('string')
df_players['National_Position'] = df_players['National_Position'].astype('string')
df_players['Club_Position'] = df_players['Club_Position'].astype('string')
df_players['Preffered_Foot'] = df_players['Preffered_Foot'].astype('string')
df_players['Work_Rate'] = df_players['Work_Rate'].astype('string')

Como podemos apreciar, podemos definir dos clases, los jugadores que son seleccionados y tuvieron una posición en un equipo y los que no. Se puede asegurar que las clases se encuentran desbalanceadas, debido a que hay solo 1075 seleccionados y 16513 que no lo fueron.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores.

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [X] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [X] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [X] Crear 3 ``pipelines`` con diferentes clasificadores.
- [X] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [X] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

Antes de empezar el escalamiento de las variables, se eliminará una variable que no tienen sentido que aportan en la predicción. Esta es el nombre de la persona. A esto hay que sumarle si o si la eliminación de la columna 'National_Position', debido a que la clase 'es_seleccionado' es una función biyectiva a esta columna, además de que ambas representan lo mismo, por lo que cometeríamos un error grave si la mantenemos esta variable para la predicción (data leakage)

In [ ]:
df_players.columns

Index(['Name', 'Nationality', 'National_Position', 'Club_Position', 'Height',
       'Weight', 'Preffered_Foot', 'Age', 'Work_Rate', 'Weak_foot',
       'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys', 'es_seleccionado'],
      dtype='object')

In [ ]:
#dropeo de la columna correspondiente al nombre y a la posición en la selección
df_players=df_players.drop(['Name', 'National_Position'], axis=1)

#variables numericas
variables_numericas=[
    'Height',
    'Weight',
    'Age',
    'Ball_Control',
    'Dribbling',
    'Marking',
    'Sliding_Tackle',
    'Standing_Tackle',
    'Aggression',
    'Reactions',
    'Interceptions',
    'Vision',
    'Composure',
    'Crossing',
    'Short_Pass',
    'Long_Pass',
    'Acceleration',
    'Speed',
    'Stamina',
    'Strength',
]

#variables categoricas
variables_categoricas=[
    'Nationality',
    'Club_Position',
    'Preffered_Foot',
    'Work_Rate',
    'Weak_foot',
    'Skill_Moves',
    'Balance', 'Agility', 'Jumping',
    'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
    'Freekick_Accuracy', 'Penalties', 'Volleys'
]

In [ ]:
numerical_transformer = StandardScaler()
ordinal_transformer = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

In [ ]:
#Realizamos One Hot Encoder a las variablees ordinales
column_transformer = ColumnTransformer(
    transformers=[
      ('ord_transform',ordinal_transformer, variables_categoricas),
      ('num_transform',numerical_transformer, variables_numericas),

    ],
    remainder='passthrough'
)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn import svm
clf1 = Pipeline(
    [("preprocesamiento", column_transformer), ("linearsvc", LinearSVC(random_state=0, tol=1e-5))]
)

clf2 = Pipeline(
    [("preprocesamiento", column_transformer), ("knn", KNeighborsClassifier(n_neighbors=3))]
)

clf3 = Pipeline(
    [("preprocesamiento", column_transformer), ("svm",svm.SVC())]
)

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [X] Separar el conjunto de datos en entrenamiento y de prueba.
- [X] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

features = df_players.drop(columns=["es_seleccionado"])
labels = df_players.loc[:, "es_seleccionado"]


X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.33, shuffle=True, stratify=labels, random_state=42
)

In [ ]:
from sklearn.pipeline import Pipeline

clf1 = clf1.fit(X_train, y_train)
clf2 = clf2.fit(X_train, y_train)
clf3 = clf3.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
y_pred_clf1 = clf1.predict(X_test)
y_pred_clf2 = clf2.predict(X_test)
y_pred_clf3 = clf3.predict(X_test)

### Linear SVC

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred_clf1))
print(classification_report(y_test, y_pred_clf1))

[[5293  157]
 [ 202  153]]
              precision    recall  f1-score   support

           0       0.96      0.97      0.97      5450
           1       0.49      0.43      0.46       355

    accuracy                           0.94      5805
   macro avg       0.73      0.70      0.71      5805
weighted avg       0.93      0.94      0.94      5805



### KNN

In [ ]:
print(confusion_matrix(y_test, y_pred_clf2))
print(classification_report(y_test, y_pred_clf2))

[[5379   71]
 [ 314   41]]
              precision    recall  f1-score   support

           0       0.94      0.99      0.97      5450
           1       0.37      0.12      0.18       355

    accuracy                           0.93      5805
   macro avg       0.66      0.55      0.57      5805
weighted avg       0.91      0.93      0.92      5805



### SVM

In [ ]:
print(confusion_matrix(y_test, y_pred_clf3))
print(classification_report(y_test, y_pred_clf3))

[[5450    0]
 [ 352    3]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5450
           1       1.00      0.01      0.02       355

    accuracy                           0.94      5805
   macro avg       0.97      0.50      0.49      5805
weighted avg       0.94      0.94      0.91      5805



**Respuesta**:
Como podemos apreciar en sus respectivas métricas,lastimósamente no se obtienen buenos resultados ante la mutilización de estos clasificadores para predecir si un jugador será seleccionado o no.
Dado el cómo se desarrolló este caso, el accuracy representa la proporción de resultados correctos. La precisión la proporción de resultados positivos en comparación a los resultados positivos existentes entre verdaderos positivos y falsos positivos. Finalmente el recall sería la división del número de verdaderos seleccionados entre la suma de verdaderos seleccionados y falsos seleccionados.


## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas.

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características.

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF']
central_ataque = ['RW', 'CAM', 'LW']
central = ['RM', 'CM', 'LM']
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [ ]:
posiciones = {'ST': 'ataque',
              'CF': 'ataque',
              'RW': 'central_ataque',
              'CAM': 'central_ataque',
              'LW': 'central_ataque',
              'RM': 'central',
              'CM': 'central',
              'LM': 'central',
              'RWB': 'central_defensa',
              'CDM': 'central_defensa',
              'LWB': 'central_defensa',
              'RB': 'defensa',
              'CB': 'defensa',
              'LB': 'defensa',
              'GK': 'arquero'
              }

In [ ]:
df_players['Simp_Position'] = df_players['Club_Position'].map(posiciones)
df_players.dropna(subset=['Simp_Position'], inplace=True)
df_players['Simp_Position']

0        central_ataque
1        central_ataque
2        central_ataque
3                ataque
4               arquero
              ...      
17298           defensa
17301           defensa
17310           central
17312           defensa
17442           defensa
Name: Simp_Position, Length: 3939, dtype: object

In [ ]:
counts = df_players['Simp_Position'].value_counts()
counts

defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: Simp_Position, dtype: int64

In [ ]:
#Pipeline
#dropeo de la columna correspondiente a la posición dentro del club, misma razón que la anterior
df_players=df_players.drop(['Club_Position'], axis=1)

#variables numericas
variables_numericas=[
    'Height',
    'Weight',
    'Age',
    'Ball_Control',
    'Dribbling',
    'Marking',
    'Sliding_Tackle',
    'Standing_Tackle',
    'Aggression',
    'Reactions',
    'Interceptions',
    'Vision',
    'Composure',
    'Crossing',
    'Short_Pass',
    'Long_Pass',
    'Acceleration',
    'Speed',
    'Stamina',
    'Strength',
]

#variables categoricas
variables_categoricas=[
    'Nationality',
    'Preffered_Foot',
    'Work_Rate',
    'Weak_foot',
    'Skill_Moves',
    'Balance', 'Agility', 'Jumping',
    'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
    'Freekick_Accuracy', 'Penalties', 'Volleys'
]

In [ ]:
#Realizamos One Hot Encoder a las variablees ordinales
column_transformer = ColumnTransformer(
    transformers=[
      ('ord_transform',ordinal_transformer, variables_categoricas),
      ('num_transform',numerical_transformer, variables_numericas),

    ],
    remainder='passthrough'
)

In [ ]:
clf1 = Pipeline(
    [("preprocesamiento", column_transformer), ("linearsvc", LinearSVC(random_state=0, tol=1e-5))]
)

clf2 = Pipeline(
    [("preprocesamiento", column_transformer), ("knn", KNeighborsClassifier(n_neighbors=3))]
)

clf3 = Pipeline(
    [("preprocesamiento", column_transformer), ("svm",svm.SVC())]
)

In [ ]:
features = df_players.drop(columns=['Simp_Position'])
labels = df_players.loc[:, 'Simp_Position']


X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.33, shuffle=True, stratify=labels, random_state=42
)

In [ ]:
clf1 = clf1.fit(X_train, y_train)
clf2 = clf2.fit(X_train, y_train)
clf3 = clf3.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
y_pred_clf1 = clf1.predict(X_test)
y_pred_clf2 = clf2.predict(X_test)
y_pred_clf3 = clf3.predict(X_test)

## Linear SVC

In [ ]:
print(confusion_matrix(y_test, y_pred_clf1))
print(classification_report(y_test, y_pred_clf1))

[[208   0   0   0   0   1]
 [  0  97  30  15   0   0]
 [  3  17 132  90  13  44]
 [  0  26  89  62   7   8]
 [  1   0  12   1  16  39]
 [  0   2  33   7  33 314]]
                 precision    recall  f1-score   support

        arquero       0.98      1.00      0.99       209
         ataque       0.68      0.68      0.68       142
        central       0.45      0.44      0.44       299
 central_ataque       0.35      0.32      0.34       192
central_defensa       0.23      0.23      0.23        69
        defensa       0.77      0.81      0.79       389

       accuracy                           0.64      1300
      macro avg       0.58      0.58      0.58      1300
   weighted avg       0.63      0.64      0.63      1300



### KNN

In [ ]:
print(confusion_matrix(y_test, y_pred_clf2))
print(classification_report(y_test, y_pred_clf2))

[[209   0   0   0   0   0]
 [  0 107  25   9   0   1]
 [  0  45 139  65   2  48]
 [  0  39  98  45   3   7]
 [  0   1  17   2  10  39]
 [  0   1  24   6   9 349]]
                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       209
         ataque       0.55      0.75      0.64       142
        central       0.46      0.46      0.46       299
 central_ataque       0.35      0.23      0.28       192
central_defensa       0.42      0.14      0.22        69
        defensa       0.79      0.90      0.84       389

       accuracy                           0.66      1300
      macro avg       0.60      0.58      0.57      1300
   weighted avg       0.64      0.66      0.64      1300



### SVM


In [ ]:
print(confusion_matrix(y_test, y_pred_clf3))
print(classification_report(y_test, y_pred_clf3))

[[209   0   0   0   0   0]
 [  0 116  21   4   0   1]
 [  0  15 187  45   5  47]
 [  0  17 107  54   4  10]
 [  0   0  13   3   9  44]
 [  0   0   9   0   3 377]]
                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       209
         ataque       0.78      0.82      0.80       142
        central       0.55      0.63      0.59       299
 central_ataque       0.51      0.28      0.36       192
central_defensa       0.43      0.13      0.20        69
        defensa       0.79      0.97      0.87       389

       accuracy                           0.73      1300
      macro avg       0.68      0.64      0.64      1300
   weighted avg       0.71      0.73      0.71      1300



**Respuesta**:

Podemos apreciar que dados los 3 modelos, lo que es más fácil es determinar a quienes serán los porteros, presentando cero errores en el proceso. POsicionándonos en los resultados del SVM, dado a que obtuvo mejores métricas, podemos apreciar que los defensass y atacantes son los que mejor calzarían con su posición, sin embargo, la mayoría de los centrales estarían mal posicionados o no pudieron ser bien catalogados como tales, quizás realmente desempeñarían mejor su puesto en otras posiciones o bien, el análisis no es lo suficientemente robusto como para asegurar qué representa un central, dado a que debe tener características de defensa como de atacante, además de estar dividido en 3 subcategorías.

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

In [173]:
df_players_2 = pd.read_csv('/content/drive/MyDrive/lab_progra/stats_players.csv')
pd.set_option('display.max_columns', None)
df_players_2

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,Ball_Control,Dribbling,Marking,Sliding_Tackle,Standing_Tackle,Aggression,Reactions,Interceptions,Vision,Composure,Crossing,Short_Pass,Long_Pass,Acceleration,Speed,Stamina,Strength,Balance,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,5,93,92,22,23,31,63,96,29,85,86,84,83,77,91,92,92,80,63,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,4,95,97,13,26,28,48,95,22,90,94,77,88,87,92,87,74,59,95,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,5,95,96,21,33,24,56,88,36,80,80,75,81,75,93,90,79,49,82,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,4,91,86,30,38,45,78,93,41,84,83,77,83,64,88,77,89,76,60,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,1,48,30,10,11,10,29,85,30,70,70,15,55,59,58,61,44,83,35,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,1,12,11,12,12,13,25,48,12,37,35,12,22,19,25,21,17,52,52,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,1,13,11,11,12,11,15,43,12,27,22,14,27,26,16,19,21,42,45,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,2,44,39,40,29,35,44,42,38,41,35,35,48,42,62,60,64,34,83,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,1,17,13,11,11,14,23,46,12,13,28,15,20,15,42,40,40,61,60,34,48,15,23,14,12,13,12,24,12


In [174]:
df_sueldos = pd.read_csv('/content/drive/MyDrive/lab_progra/salarios.csv')
pd.set_option('display.max_columns', None)
df_sueldos = df_sueldos.rename(columns={'Player': 'Name'})
df_sueldos

,Unnamed: 0,Name,Weekly Salary
0,1528,Lionel Messi,1538905.0
1,1006,Cristiano Ronaldo,1248536.0
2,1529,Antoine Griezmann,997032.0
3,2004,Neymar,797726.0
4,1530,Gareth Bale,607706.0
...,...,...,...
2529,1527,Simone Ghidotti,283.0
2530,2539,Valentin Gendrey,100.0
2531,1001,John Yeboah,70.0
2532,1000,Antonis Aidonis,70.0


In [175]:
df_salarios =  df_sueldos.merge(df_players_2, how='inner', on='Name')
df_salarios

,Unnamed: 0,Name,Weekly Salary,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,Ball_Control,Dribbling,Marking,Sliding_Tackle,Standing_Tackle,Aggression,Reactions,Interceptions,Vision,Composure,Crossing,Short_Pass,Long_Pass,Acceleration,Speed,Stamina,Strength,Balance,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,1528,Lionel Messi,1538905.0,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,4,95,97,13,26,28,48,95,22,90,94,77,88,87,92,87,74,59,95,90,68,71,85,95,88,89,90,74,85
1,1006,Cristiano Ronaldo,1248536.0,Portugal,LS,LW,185,80,Right,32,High / Low,4,5,93,92,22,23,31,63,96,29,85,86,84,83,77,91,92,92,80,63,90,95,85,92,93,90,81,76,85,88
2,1529,Antoine Griezmann,997032.0,France,CAM,RS,176,67,Left,26,High / Medium,3,4,86,87,23,11,22,69,90,35,76,81,82,79,75,87,86,75,61,80,90,87,80,81,88,82,84,75,71,87
3,2004,Neymar,797726.0,Brazil,LW,LW,174,68,Right,25,High / Medium,5,5,95,96,21,33,24,56,88,36,80,80,75,81,75,93,90,79,49,82,96,61,62,78,89,77,79,84,81,83
4,1530,Gareth Bale,607706.0,Wales,RS,RW,183,74,Left,27,High / Medium,3,4,88,89,51,52,55,65,87,59,79,85,87,86,80,93,95,78,80,65,77,85,86,91,87,90,86,85,76,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1861,986,Michael Langer,903.0,Austria,NaN,GK,194,92,Right,32,Medium / Medium,3,1,21,11,15,13,12,35,62,15,59,41,13,29,25,31,32,32,83,34,22,50,14,15,14,13,12,12,13,11
1862,2536,Nicolas Lemaître,859.0,France,NaN,Res,185,83,Right,20,Medium / Medium,2,1,24,20,14,15,11,18,46,24,19,23,22,28,19,34,32,28,26,37,24,32,23,23,17,15,19,14,22,17
1863,1523,Andrea Pinamonti,805.0,Italy,NaN,Res,181,72,Right,17,Medium / Low,3,2,68,62,22,19,19,32,52,16,45,51,40,53,36,55,55,55,55,65,65,65,62,57,74,60,43,43,69,43
1864,989,Tim Wiesner,674.0,Germany,NaN,Res,192,87,Right,20,Medium / Medium,3,1,19,12,12,14,15,26,51,14,31,30,12,22,20,36,34,23,66,35,39,57,16,19,13,15,18,16,20,14


In [176]:
df_salarios.columns

Index(['Unnamed: 0', 'Name', 'Weekly Salary', 'Nationality',
       'National_Position', 'Club_Position', 'Height', 'Weight',
       'Preffered_Foot', 'Age', 'Work_Rate', 'Weak_foot', 'Skill_Moves',
       'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys'],
      dtype='object')

In [177]:
#Limpiamos el dataset
df_salarios = df_salarios.drop(['Unnamed: 0', 'Name', 'National_Position',], axis=1)

#variables numericas
variables_numericas=[
    'Height',
    'Weight',
    'Age',
    'Ball_Control',
    'Dribbling',
    'Marking',
    'Sliding_Tackle',
    'Standing_Tackle',
    'Aggression',
    'Reactions',
    'Interceptions',
    'Vision',
    'Composure',
    'Crossing',
    'Short_Pass',
    'Long_Pass',
    'Acceleration',
    'Speed',
    'Stamina',
    'Strength',
]

#variables categoricas
variables_categoricas=[
    'Nationality',
    'Club_Position',
    'Preffered_Foot',
    'Work_Rate',
    'Weak_foot',
    'Skill_Moves',
    'Balance', 'Agility', 'Jumping',
    'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
    'Freekick_Accuracy', 'Penalties', 'Volleys'
]

In [200]:
#Realizamos One Hot Encoder a las variablees ordinales
column_transformer = ColumnTransformer(
    transformers=[
      ('ord_transform',ordinal_transformer, variables_categoricas),
      ('num_transform',numerical_transformer, variables_numericas),

    ],
    remainder='passthrough'
)

In [221]:
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
clf_reg = Pipeline(
    [
        ("preprocesamiento", column_transformer),
        ('pca', PCA(n_components=8)),
        ("ridge", Ridge(alpha=1))
    ]
)

In [222]:
#Ordenamos los datos
X = df_salarios.drop(['Weekly Salary'], axis=1)
y = df_salarios['Weekly Salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [223]:
#Entrenamos
clf_reg = clf_reg.fit(X_train, y_train)

In [224]:
#Evaluamos
r2_score = clf_reg.score(X_test, y_test)
print("(R^2):", r2_score)

(R^2): 0.2135013260811771


**Respuesta**:

Dada la cantidad de componentes existentes, fue necesario realizar PCA, si bien se probó con distintos valores y  distintos modelos, este fue el mejor resultado que se pudo obtener, con un R^2 de 0.25.
Este coeficiente indica la proporción de la varianza de la variable dependiente que se entiende gracias al modelo, un valor cercano a 1 nos dice que todos los valores predichos coincidirían con los observados, pero puedo implicar un sobreajuste.

Para mejorar los resultados puedwe ser posiblke tener unos datos con menos variabilidad, lo pueden ser el tiempo en cancha, edad de debut como profesional, equipo en el que juega, entre otros que pueden ayudar a entender mejor al modelo.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>